<a href="https://colab.research.google.com/github/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021/blob/main/examples/DQN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install git+https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git to /tmp/pip-req-build-99o0ht95
  Running command git clone -q https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git /tmp/pip-req-build-99o0ht95
  Created wheel for RLalgos: filename=RLalgos-0.2.1-py3-none-any.whl size=10833 sha256=385beb54139427db72d8a8ebb06c2dfa812a009c764e2ee8740cc6f74609f3a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-dfxah5s5/wheels/4c/56/f2/8ddd79049992cb6f5396813f1f0ab97eac7ab2633c7c9eb79d
Successfully built RLalgos
  Attempting uninstall: RLalgos
    Found existing installation: RLalgos 0.2.0
    Uninstalling RLalgos-0.2.0:
      Successfully uninstalled RLalgos-0.2.0


In [11]:
import gym
import numpy as np
import os
import torch
import math

In [6]:
from RLalgos import DQN

# Config

In [12]:
class Config(object):
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Main agent variables
        self.GAMMA=0.99
        self.LR=1e-3
        
        # Epsilon variables
        self.epsilon_start    = 1.0
        self.epsilon_final    = 0.01
        self.epsilon_decay    = 10000
        self.epsilon_by_sample = lambda sample_idx: config.epsilon_final + (config.epsilon_start - config.epsilon_final) * math.exp(-1. * sample_idx / config.epsilon_decay)

        # Memory
        self.TARGET_NET_UPDATE_FREQ = 1000
        self.EXP_REPLAY_SIZE = 10000
        self.BATCH_SIZE = 64

        # Learning control variables
        self.LEARN_START = 1000
        self.MAX_SAMPLES = 50000
        self.UPDATE_FREQ = 1

        # Data logging parameters
        self.ACTION_SELECTION_COUNT_FREQUENCY = 1000
        
        
config = Config()

# Training

In [13]:

log_dir = "/tmp/gym/"

if os.path.exists(log_dir):
    os.system(f'rm -rf {log_dir}')
os.makedirs(log_dir)


env_id = 'CartPole-v0'
env    = gym.make(env_id)
    
model  = DQN(env=env, config=config, log_dir=log_dir)

episode_reward = 0
episode_reward_record = []
observation = env.reset()
episode = 0

for sample_idx in range(1, config.MAX_SAMPLES + 1):
    
    epsilon = config.epsilon_by_sample(sample_idx)

    action = model.get_action(observation, epsilon)
    # Log action selection
    model.save_action(action, sample_idx)

    prev_observation=observation
    observation, reward, done, _ = env.step(action)
    observation = None if done else observation

    model.update(prev_observation, action, reward, observation, sample_idx)
    episode_reward += reward

    if done:
        observation = env.reset()
        model.save_reward(episode_reward)
        episode_reward_record.append(episode_reward)

        episode += 1
        if episode % 100 == 0:
            print(f'{episode}th episode, mean eposide reward: {np.mean(episode_reward_record)}')
        episode_reward = 0

        
        

    

        

model.save_w()
env.close()

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

100th episode, mean eposide reward: 25.05
200th episode, mean eposide reward: 33.01
300th episode, mean eposide reward: 70.48
400th episode, mean eposide reward: 90.6975
